#  Install package

In [1]:
# ! pip install pyvis
# ! pip install networkx

# Import package

In [2]:
import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
from pyvis.network import Network
import openai
from IPython.display import display, HTML, Code
from IPython.display import clear_output

# Define Solution class
Please run the following cell to define the functions

In [3]:
import LLM_Geo_Constants as constants
import helper

from LLM_Geo_kernel import Solution

%load_ext autoreload
%autoreload 2

# Demonstration Cases

## Input task and data desciption

In [4]:
# # Case 4: Hospital_accessibility
# task_name ='Hospital_accessibility'
# TASK = r'''
# For each zipcode area in South Carolina (SC), calculate the distance from the centroid of the zipcode area to its nearest hospital, and then create a choropleth distance map of zipcode area polygons (unit: km), also show the hospital.
# '''

# DATA_LOCATIONS = [
# r"SC zipcode boundary shapefile: https://github.com/GIBDUSC/test/raw/master/sc_zip_boundary.zip, the map projection is WGS1984.",
# r"SC hospitals:  https://github.com/gladcolor/spatial_data/raw/master/South_Carolina/SC_hospitals_with_emergency_room_cleaned.csv, location columns: longitude in 'POINT_X' column, latitude in 'POINT_Y' column.",          
# ]


## Case 5: Census API
# task_name ='Census_API'
# TASK = r''' Show the spatial distribution of the county level median income in the contigous US. Set figure size to (25,15)
# '''

# DATA_LOCATIONS = [
#       "You can use the Census API.",
# ]


# DATA_LOCATIONS = [
# '''Census API key: ae7be70727932dd6aed257692de3f344365d0678'''              
# ]


# # Case 1: population living near hazardous wastes

# task_name ='Resident_at_risk_counting'

# TASK = r"""1) Find out the total population that lives within a tract that contain hazardous waste facilities. The study area is North Carolina, US.
# 2) Generate a map to show the spatial distribution of population at the tract level and highlight the borders of tracts that have hazardous waste facilities.
# """

# DATA_LOCATIONS = ["NC hazardous waste facility ESRI shape file location: https://github.com/gladcolor/LLM- Geo/raw/master/overlay_analysis/Hazardous_Waste_Sites.zip.",
#                   "NC tract boundary shapefile location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_shp_37.zip. The tract id column is 'Tract'.",
#                   "NC tract population CSV file location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv. The population is stored in 'TotalPopulation' column. The tract ID column is 'GEOID'."
#                  ]





# # Case 3: COVID-19 prevalence trend
# task_name ='COVID_death_rate'
# TASK = r'''1) Draw a map to show the death rate (death/case) of COVID-19 among the countiguous US counties. Use the accumulated COVID-19 data of 2020.12.31 to compute the death rate. Use scheme ='quantiles' when plotting the map.  Set map projection to 'Conus Albers'. Set map size to 15*10 inches.  
# 2) Draw a scatter plot to show the correlation and trend line of the death rate with the senior resident rate, including the r-square and p-value. Set data point transparency to 50%, regression line as red.  Set figure size to 15*10 inches.  
# '''


# DATA_LOCATIONS = [
#                   r"COVID-19 data case in 2020 (county-level): https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv. This data is for daily accumulated COVID cases and deaths for each county in the US. There are 5 columns: date (format: 2021-02-01), county, state, fips, cases, deaths. ",   
#                   r"Contiguous US county boundary (ESRI shapefile): https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip. The county FIPS column is 'GEOID'. ",
#                   r"Census data (ACS2020): https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county.csv. THe needed columns are: 'FIPS', 'Total Population', 'Total Population: 65 to 74 Years', 'Total Population: 75 to 84 Years', 'Total Population: 85 Years and Over'. Drop rows with NaN cells after loading the used columns.",
#                  ]


# Case 2: France_mobility_changes_2020
task_name ='France_mobility_changes_2020'
TASK = r'''
1) Show the 2020 human mobility monthly change rates of each administrative regions in a France choropleth map. Each month is a sub-map in a map matrix，12 months in total. All monthly maps need to use the same colorbar range. The base of the change rate is January 2020. 
2) Draw a line chart to show the monthly change rate trends of all administrative regeions. Each region is a line (the region name is the legend), the x-axis is 2020 months.
'''

DATA_LOCATIONS = ["ESRI shapefile for France administrative regions:" + \
                  "https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip. " + \
                  "The 'GID_1' column is the administrative region code, 'NAME_1' column is the administrative region name.",
                  "REST API url with parameters for daily human mobility data access:" + \
                  "http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020." + \
                  "The response is in CSV format. There are three columns in the response: " + \
                  "place,date (format:2020-01-07), and intra_movement. 'place' column is the administractive region code of every country;" + \
                  "codes for France administrative regions start with 'FRA'. Use the total intra_movement of the month as the montly mobility.",
                 ]



save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)

# create graph
model=r"gpt-4"
solution = Solution(
                    task=TASK,
                    task_name=task_name,
                    save_dir=save_dir,
                    data_locations=DATA_LOCATIONS,
                    model=model,
                    )
print("Prompt to get solution graph:\n")
print(solution.graph_prompt)

Prompt to get solution graph:

Your role: A professional Geo-information scientist and developer good at Python. 

Your task: Generate a graph (data structure) only, whose nodes are (1) a series of consecutive steps and (2) data to solve this question:  
 
1) Show the 2020 human mobility monthly change rates of each administrative regions in a France choropleth map. Each month is a sub-map in a map matrix，12 months in total. All monthly maps need to use the same colorbar range. The base of the change rate is January 2020. 
2) Draw a line chart to show the monthly change rate trends of all administrative regeions. Each region is a line (the region name is the legend), the x-axis is 2020 months.
 

Your reply needs to meet these requirements: 
 1. Think step by step.
2. Steps and data (both input and output) form a graph stored in NetworkX. Disconnected components are NOT allowed.
3. Each step is a data process operation: the input can be data paths or variables, and the output can be da

## Get graph code from GPT API

In [5]:
response_for_graph = solution.get_LLM_response_for_graph() 
solution.graph_response = response_for_graph
solution.save_solution()

clear_output(wait=True)
display(Code(solution.code_for_graph, language='python'))

import networkx as nx

G = nx.DiGraph()

# 1 Load France region shapefile
G.add_node("France_shp_url", node_type="data", data_path="https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip", description="ESRI shapefile for France administrative regions")
G.add_node("load_France_shp", node_type="operation", description="Load France region shapefile")
G.add_edge("France_shp_url", "load_France_shp")
G.add_node("France_gdf", node_type="data", description="France region GeoDataFrame")
G.add_edge("load_France_shp", "France_gdf")

# 2 Load movement data 
G.add_node("api_url_param", node_type="data", data_path="http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020", description="REST API url with parameters for daily human mobility data")
G.add_node("load_movement_data", node_type="operation", description="Load and filter movement data")
G.add_edge("api_url_param", "load_movement_data")
G.add_node("movement_data", node_type="data", description="Movement Data in DataFrame format")
G.add_edge("load_movement_data", "movement_data")

# 3 Smash join region and movement data
G.add_node("join_region_movement", node_type="operation", description="Smash join region data and movement data")
G.add_edge("France_gdf", "join_region_movement")
G.add_edge("movement_data", "join_region_movement")
G.add_node("joined_data", node_type="data", description="Smashed joined region and movement data")
G.add_edge("join_region_movement", "joined_data")

# 4 Calculate monthly change rates
G.add_node("calc_change_rate", node_type="operation", description="Compute mobility monthly change rates")
G.add_edge("joined_data", "calc_change_rate")
G.add_node("change_rate", node_type="data", description="Mobility monthly change rates")
G.add_edge("calc_change_rate", "change_rate")

# 5 Generate choropleth map
G.add_node("create_choropleth", node_type="operation", description="Create choropleth map with the change rates")
G.add_edge("change_rate", "create_choropleth")
G.add_node("choropleth_map", node_type="data", description="Choropleth maps for each month")
G.add_edge("create_choropleth", "choropleth_map")

# 6 Generate line chart
G.add_node("create_line_chart", node_type="operation", description="Create line chart for monthly change rates for each region")
G.add_edge("change_rate", "create_line_chart")
G.add_node("line_chart", node_type="data", description="Line Chart of monthly change")
G.add_edge("create_line_chart", "line_chart")

# save graph
nx.write_graphml(G, 'E:\\Research\\LLM-Geo\\France_mobility_changes_2020\\France_mobility_changes_2020.graphml')

## Execute code to generate the solution graph

In [6]:
exec(solution.code_for_graph)
solution_graph = solution.load_graph_file()

# Show the graph
G = nx.read_graphml(solution.graph_file)  
nt = helper.show_graph(G)
html_name = os.path.join(os.getcwd(), solution.task_name + '.html')  
# HTML file should in the same directory. See:
# https://stackoverflow.com/questions/65564916/error-displaying-pyvis-html-inside-jupyter-lab-cell
nt.show(name=html_name)
# html_name

E:\Research\LLM-Geo\France_mobility_changes_2020.html


## Generate prompts and code for operations (functions)

In [7]:
operations = solution.get_LLM_responses_for_operations(review=True)
solution.save_solution()

all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])

clear_output(wait=True)
display(Code(all_operation_code_str, language='python'))

import geopandas as gpd

def load_France_shp(France_shp_url='https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip'):
    '''
    Description: Load France region shapefile
    Input:
        France_shp_url (str): URL of the ESRI Shapefile for France administrative regions
    Output:
        France_gdf (geopandas.GeoDataFrame): France region GeoDataFrame. Contains the geometries of France administrative regions.
    '''
    # Load ESRI Shapefile from the given URL
    France_gdf = gpd.read_file(France_shp_url)
    # Convert the map projection to the commonly used map projection (EPSG: 4326 - WGS84)
    France_gdf = France_gdf.to_crs(epsg=4326)
    return France_gdf
import pandas as pd
import requests
from io import StringIO

def load_movement_data(api_url_param='http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020'):
    """
    Description: 
        Load and filter movement data from a REST API using provided URL link.

    Arguments:
        api_url_param: REST API url with parameters for daily human mobility data.

    Returns:
        movement_data: DataFrame format for loaded and filtered movement data.
    """
    response = requests.get(api_url_param)
    movement_data_source = pd.read_csv(StringIO(response.text))

    # Filtering French administrative regions which codes start with 'FRA'
    movement_data = movement_data_source[movement_data_source['place'].str.startswith('FRA')]

    # Changing date format to Year-Month to assist in further analyses
    movement_data['date'] = pd.to_datetime(movement_data['date']).dt.to_period('M')

    # Summing up 'intra_movement' for each month
    movement_data = movement_data.groupby(['place', 'date'])['intra_movement'].sum().reset_index()

    movement_data['place'] = movement_data['place'].astype(str)

    return movement_data
def join_region_movement(France_gdf, movement_data):
    """
    Description: 
        Smash join region data and movement data.

    Arguments:
        France_gdf (geopandas.GeoDataFrame): France region GeoDataFrame. Contains the geometries of France administrative regions.
        movement_data (pandas.DataFrame): Movement data in DataFrame format.

    Returns:
        joined_data (pandas.DataFrame): Smashed joined region and movement data.
    """
    # Change 'GID_1' column to string type for proper join.
    France_gdf['GID_1'] = France_gdf['GID_1'].astype(str)
    
    # Smash join region data and movement data.
    joined_data = France_gdf.merge(movement_data, left_on='GID_1', right_on='place')

    # Removing duplicates from joined data.
    joined_data.drop_duplicates(inplace=True)

    return joined_data
def calc_change_rate(joined_data):
    """
    Description: 
        Compute mobility monthly change rates using the joined data of region and movement.

    Arguments:
        joined_data (pandas.DataFrame): Pandas DataFrame including the information of administrative regions and their monthly summarised mobility data.

    Returns:
        change_rate (pandas.DataFrame): DataFrame including the monthly change rates of human mobility for all administrative regions in 2020. 
    """
    # Calculating the monthly change rates based on the base month (January 2020)
    # We create a pivot table for easy calculation
  
    pivot_table = joined_data.pivot_table(index='GID_1', 
                                          columns='date', 
                                          values='intra_movement').reset_index()
   
    # Using the .pct_change() function to calculate change rates, and transform it it to 'long' format using .melt()
    change_rate = pivot_table.loc[:, '2020-01':'2020-12'].pct_change(axis=1).melt(ignore_index=False).reset_index()

    # Adding the 'intra_movement' column with new change_rate values
    change_rate['intra_movement'] = change_rate['value']

    return change_rate
import matplotlib.pyplot as plt
fr

## Generate prompts and code for assembly program

In [8]:
assembly_LLM_response = solution.get_LLM_assembly_response(review=True)
solution.assembly_LLM_response = assembly_LLM_response
solution.save_solution()

clear_output(wait=True)
display(Code(solution.code_for_assembly, language='python'))

import geopandas as gpd
import pandas as pd
import requests
from io import StringIO
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


def load_France_shp(France_shp_url='https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip'):
    France_gdf = gpd.read_file(France_shp_url)
    France_gdf = France_gdf.to_crs(epsg=4326)
    return France_gdf


def load_movement_data(api_url_param='http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020'):
    response = requests.get(api_url_param)
    movement_data_source = pd.read_csv(StringIO(response.text))
    movement_data = movement_data_source[movement_data_source['place'].str.startswith('FRA')]
    movement_data['date'] = pd.to_datetime(movement_data['date']).dt.to_period('M')
    movement_data = movement_data.groupby(['place', 'date'])['intra_movement'].sum().reset_index()
    movement_data['place'] = movement_data['place'].astype(str)
    return movement_data


def join_region_movement(France_gdf, movement_data):
    France_gdf['GID_1'] = France_gdf['GID_1'].astype(str)
    joined_data = France_gdf.merge(movement_data, left_on='GID_1', right_on='place')
    joined_data.drop_duplicates(inplace=True)
    return joined_data


def calc_change_rate(joined_data):
    pivot_table = joined_data.pivot_table(index='GID_1', 
                                          columns='date', 
                                          values='intra_movement').reset_index()
    change_rate = pivot_table.loc[:, '2020-01':'2020-12'].pct_change(axis=1).melt(ignore_index=False).reset_index()
    change_rate['intra_movement'] = change_rate['value']
    return change_rate


def create_choropleth(France_gdf, change_rate):
    pivot_change_rate = pd.pivot_table(change_rate, index='GID_1', columns='date', values='intra_movement')
    vmin, vmax = pivot_change_rate.min().min(), pivot_change_rate.max().max()
    fig, axs = plt.subplots(4, 3, figsize=(15, 20))

    for ax, (month, values) in zip(axs.flatten(), pivot_change_rate.iteritems()):   
        merged = France_gdf.set_index('GID_1').join(values)
        merged = merged.reset_index()
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        merged.plot(column=month, cmap='viridis', linewidth=0.8, 
                    ax=ax, edgecolor='0.8', vmin=vmin, vmax=vmax, cax=cax, legend=True)
        ax.axis('off')
        ax.set_title(month)
        
    plt.tight_layout()
    choropleth_map = plt.gcf()

    return choropleth_map


def create_line_chart(change_rate):
    pivoted = change_rate.pivot(index='date', columns='GID_1', values='intra_movement')
    plt.figure(figsize=(10,8))
    for column in pivoted.columns:
        plt.plot(pivoted.index.strftime('%Y-%m'), pivoted[column], label=column)
    plt.title('Monthly Change Rate Trends of All Administrative Regions', fontsize=14)
    plt.xlabel('Dates', fontsize=12)
    plt.ylabel('Change Rate', fontsize=12)
    plt.legend(loc='upper right', fontsize=10)
    line_chart = plt.gcf()

    return line_chart 


def assembely_solution():
    France_gdf = load_France_shp()
    movement_data = load_movement_data()
    joined_data = join_region_movement(France_gdf, movement_data)
    change_rate = calc_change_rate(joined_data)
    choropleth_map = create_choropleth(France_gdf, change_rate)
    choropleth_map.savefig("month_change_rate_map.png")
    line_chart = create_line_chart(change_rate)
    line_chart.savefig("month_change_rate_line_chart.png")

assembely_solution()

## Execute assembly code

In [9]:
all_code = all_operation_code_str + '\n' + solution.code_for_assembly

# display(Code(all_code, language='python'))

all_code = solution.execute_complete_program(code=all_code, try_cnt=10)



-------------- Running code (trial # 1/10) --------------


Error_info_str: 
Traceback (most recent call last):
  File "Complete program", line 259, in <module>
    assembely_solution()
  File "Complete program", line 251, in assembely_solution
    movement_data = load_movement_data()
  File "Complete program", line 191, in load_movement_data
    movement_data = movement_data_source[movement_data_source['place'].str.startswith('FRA')]
  File "e:\ProgramData\Anaconda3\envs\street_mapping_env\lib\site-packages\pandas\core\frame.py", line 3805, in __getitem__
    indexer = self.columns.get_loc(key)
  File "e:\ProgramData\Anaconda3\envs\street_mapping_env\lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    raise KeyError(key) from err
KeyError: 'place'

Sending error information to LLM for debugging...
The error message `KeyError: 'place'` indicates that the column 'place' doesn't exist in the DataFrame 'movement_data_source'. After checking the script, we found tha

Traceback (most recent call last):
  File "e:\ProgramData\Anaconda3\envs\street_mapping_env\lib\site-packages\pandas\core\indexes\base.py", line 3803, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'place'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "E:\Research\LLM-Geo\LLM_Geo_kernel.py", line 425, in execute_complete_program
    exec(compiled_code, globals())  # #pass only globals() not locals()
  File "Complete program", line 259, in <module>
    f"The ancestor function code is (need to follow the generated

In [60]:
import geopandas as gpd
import pandas as pd
import requests
from io import StringIO
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


def load_France_shp(France_shp_url='https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip'):
    France_gdf = gpd.read_file(France_shp_url)
    France_gdf = France_gdf.to_crs(epsg=4326)
    return France_gdf


def load_movement_data(api_url_param='http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020'):
    response = requests.get(api_url_param)
    movement_data_source = pd.read_csv(StringIO(response.text))
    movement_data = movement_data_source[movement_data_source['place'].str.startswith('FRA')]
    movement_data['date'] = pd.to_datetime(movement_data['date']).dt.to_period('M')
    movement_data = movement_data.groupby(['place', 'date'])['intra_movement'].sum().reset_index()
    movement_data['place'] = movement_data['place'].astype(str)
    return movement_data


def join_region_movement(France_gdf, movement_data):
    France_gdf['GID_1'] = France_gdf['GID_1'].astype(str)
    joined_data = France_gdf.merge(movement_data, left_on='GID_1', right_on='place')
    joined_data.drop_duplicates(inplace=True)
    return joined_data


def calc_change_rate(joined_data):
    pivot_table = joined_data.pivot_table(index='GID_1', 
                                          columns='date', 
                                          values='intra_movement').reset_index()
    # return pivot_table
    change_rate = pivot_table.loc[:, pd.Period('2020-01'):pd.Period('2020-12')].pct_change(axis=1).melt(ignore_index=False).reset_index()
    change_rate['intra_movement'] = change_rate['value']
    return change_rate


def create_choropleth(France_gdf, change_rate):
    pivot_change_rate = pd.pivot_table(change_rate, index='GID_1', columns='date', values='intra_movement')
    vmin, vmax = pivot_change_rate.min().min(), pivot_change_rate.max().max()
    fig, axs = plt.subplots(4, 3, figsize=(15, 20))

    for ax, (month, values) in zip(axs.flatten(), pivot_change_rate.iteritems()):   
        merged = France_gdf.set_index('GID_1').join(values)
        merged = merged.reset_index()
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        merged.plot(column=month, cmap='viridis', linewidth=0.8, 
                    ax=ax, edgecolor='0.8', vmin=vmin, vmax=vmax, cax=cax, legend=True)
        ax.axis('off')
        ax.set_title(month)
        
    plt.tight_layout()
    choropleth_map = plt.gcf()

    return choropleth_map


def create_line_chart(change_rate):
    pivoted = change_rate.pivot(index='date', columns='GID_1', values='intra_movement')
    plt.figure(figsize=(10,8))
    for column in pivoted.columns:
        plt.plot(pivoted.index.strftime('%Y-%m'), pivoted[column], label=column)
    plt.title('Monthly Change Rate Trends of All Administrative Regions', fontsize=14)
    plt.xlabel('Dates', fontsize=12)
    plt.ylabel('Change Rate', fontsize=12)
    plt.legend(loc='upper right', fontsize=10)
    line_chart = plt.gcf()

    return line_chart 


# def assembely_solution():
# France_gdf = load_France_shp()
# movement_data = load_movement_data()
# joined_data = join_region_movement(France_gdf, movement_data)
change_rate = calc_change_rate(joined_data)
change_rate
# choropleth_map = create_choropleth(France_gdf, change_rate)
# choropleth_map.savefig("month_change_rate_map.png")
# line_chart = create_line_chart(change_rate)
# line_chart.savefig("month_change_rate_line_chart.png")

C:\Users\N\AppData\Local\Temp\ipykernel_8452\1198219668.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movement_data['date'] = pd.to_datetime(movement_data['date']).dt.to_period('M')


,index,date,value,intra_movement
0,0,2020-01,NaN,NaN
1,1,2020-01,NaN,NaN
2,2,2020-01,NaN,NaN
3,3,2020-01,NaN,NaN
4,4,2020-01,NaN,NaN
...,...,...,...,...
151,8,2020-12,0.055649,0.055649
152,9,2020-12,-0.002930,-0.002930
153,10,2020-12,0.033751,0.033751
154,11,2020-12,-0.014717,-0.014717


In [58]:
# change_rate.loc[:, pd.Period('2020-01'):pd.Period('2020-12')]
print(change_rate.columns)
type(change_rate.columns[4])
# change_rate

Index(['GID_1', 2020-01, 2020-02, 2020-03, 2020-04, 2020-05, 2020-06, 2020-07,
       2020-08, 2020-09, 2020-10, 2020-11, 2020-12],
      dtype='object', name='date')


pandas._libs.tslibs.period.Period

In [59]:
change_rate

# change_rate.loc[:, '2020-01':'2020-12'].pct_change(axis=1).melt(ignore_index=False).reset_index()
change_rate.loc[:, '2020-01':'2020-12']#.pct_change(axis=1).melt(ignore_index=False).reset_index()

KeyError: '2020-01'